# Импортирование необходимых модулей

Для модуля NLTK необходима дополнительная загрузка данных для функций токенизации, лемматизации и для корпуса стоп-слов.

In [1]:
import re
import nltk
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize # см. ниже
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from joblib import Parallel, delayed # см. ниже
from unidecode import unidecode # см. ниже
from datetime import datetime # см. ниже

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nshir\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nshir\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nshir\AppData\Roaming\nltk_data...


True

# Загрузка датасета из CSV-файла

Файл должен находиться в той же папке, что и блокнот. В противном случае, необходимо указать к нему путь. Предоставленный файл содержит хедер с названиями колонок. Содержание текстовых строк соответствует кодировке ISO/IEC 8859-1, или Latin-1.

In [2]:
with open('Task_1_original_data.csv', newline='') as csvfile:
    df = pd.read_csv(csvfile, encoding='latin_1')
df = df.dropna()
df

,class,date,from,to,subject,body
0,0,2002-03-15T17:06:22,info@global-change.com,michelle.lokay@enron.com,Next Wave of Energy Trading,Energy Industry Professional: Â Global Chang...
1,0,2002-01-29T11:16:35,info@pmaconference.com,michelle.lokay@enron.com,Register for the Next TXU Capacity Auction!,Register for the next TXU Energy Capacity Auct...
2,0,2002-01-27T22:06:53,info@pmaconference.com,michelle.lokay@enron.com,Merchant Power Monthly Free Sample,.................................................
3,0,2002-03-14T07:22:17,bruno@firstconf.com,energynews@fc.ease.lsoft.com,"t,h: Eyeforenergy Update",Welcome to this week's Eyeforenergy Update. J...
4,0,2002-03-05T03:51:53,deanrogers@energyclasses.com,michelle.lokay@enron.com,"Derivatives Early Bird 'til March 11, Houston",Derivatives For Energy Professionals Two Ful...
...,...,...,...,...,...,...
31710,1,2004-06-24T04:51:30,jacob rzucidlo <lavoneaker@stalag13.com>,johnny wynott <varou@iit.demokritos.gr>,Cpu PAIN M.edICATI0N... SHIPPED to Your D00R !,\n\n arrghh wests amnstv amlsmith basus\nÂ...
31711,1,2004-06-19T23:52:54,hal leake <annettgaskell@buglover.net>,renato mooney <sigletos@iit.demokritos.gr>,Dn trouble f.r.ee,\n\n\n\n\nDn trouble f.r.ee\n\n\n\nangiospasmÂ...
31712,1,2004-06-30T06:07:33,dr collins khumalo <khumalo_20@sunumail.sn>,khumalo_20@sunumail.sn,Dr.Collins Khumalo.,\n\n\n\n\nDr.Collins Khumalo.\n\n\n\n\n\n\n\n\...
31713,1,2004-10-10T12:00:18,Customer Support <support@citibank.com>,Paliourg <paliourg@iit.demokritos.gr>,Dear customer your details have been compromised,\n\n\n\n\nDear customer your details have been...


# Предварительная обработка содержания

Определение функции для предварительной обработки. Имеет значение порядок выполняемых операций:
1. В первую очередь удаляются крупные элементы, т.е. URL ссылки и email адреса. Поскольку URL ссылки могут содеражать email адреса, они удаляются в первую очередь. URL ссылки могут как содержать компонент схемы (`http`, `https`, `ftp`) с разделителем (`://`), так и нет. Когда URL не содержит схему, он может быть практически не отличим от последовательности слов, разделенных точкой. Поэтому для удаления URL используется регулярное выражение с компонентом схемы во избежание удаления большого количества действительных слов. Для повторимости результатов задания необходимо использовать предоставленное регулярное выражение: `\b[a-z]*:\/{2}\S*\s*`.
1. Регулярные выражения для email адресов, соответствующие RFC 5322 и RFC 822, слишком громоздки и неэффективны, и их использование для практического задания нецелесообразно. Для задания предоставляется простое выражение `[^@\s]+@[^@\s]+\.[^@\s]+`.
1. Для упрощения разделения строки на слова и пунктуацию (токенизацию) используется функция `word_tokenize` из модуля `nltk`. После разделения обрабатывается отдельно каждый токен, полученный из строки.
1. Токен может быть как словом, так и знаком пунктуации, так и комбинацией символов, в том числе плохо раскодированных при обработке датасета (например, `Õåê~úßÑ`). Для определения, является ли токен состоятельным кандидатом слова, проверяется наличие хотя бы одной буквы стандартного латинского алфавита. Данный выбор вызван тем, что слова, полностью состоящие из диакритизированных латинских букв, в европейских языках отсутствуют, или очень редки, или являются служебными. Это позволит отсеять плохо раскодированные знаки и знаки пунктуации без потери большого количества действительных слов.
1. Символы отсеянных токенов-кандидатов конвертируются в стандартный латинский алфавит с помощью модуля `unidecode`.
1. Последней фильтрацией полученных токенов является проверка на стоп-слова. Для этого применяется корпус стоп-слов `stopwords` из модуля `nltk`.
1. Несмотря на то что лемматизация и вычленение корня (стемминг) — похожие методы преобразования слов, в задании предлагается использовать их совместно. В первую очередь необходимо применить лемматизацию, так как она ставит в соответствие слову лемму согласно корпусу языка, в то время как стемминг приводит слово к рудиментарному корню, не являющемуся действительным словом, на основе эвристических методов. Поэтому применение стемминга *до* лемматизация невозможно. Для обоих процедур применяется модуль `nltk`, а именно классы `WordNetLemmatizer` и `PorterStemmer`.

Также необходимо обратить внимание, что для некоторых писем содержание может полностью отсутствовать.

In [2]:
wnl = WordNetLemmatizer()
ps = PorterStemmer()
stopwords = stopwords.words()

def token_preproc(token: str):
    if re.search(r'[a-z]', token) is None:
        return ''
    else:
        # Замена диакритизированных символов стандартной латиницей
        token = unidecode(token)
        # Удаление стоп-слов
        if token in stopwords:
            return ''
        else:
            # Лемматизация
            token = wnl.lemmatize(token)
            # Стемминг
            return ps.stem(token)

def preprocess(body):
    if not pd.isnull(body):
        # Приведение к нижнему регистру
        string = body.lower()
        # Удаление URL
        string = re.sub(r'\b[a-z]*:\/{2}\S*\s*', ' ', string)
        # Удаление адресов электронной почты
        string = re.sub(r'[^@\s]+@[^@\s]+\.[^@\s]+', '', string)
        # Токенизация
        tokens = word_tokenize(string)           
        # Обработка токенов (см. выше)
        tokens = [token_preproc(t) for t in tokens]
        # Соединение токенов обратно в строку
        string = ' '.join(tokens)
        # Удаление оставшихся знаков пунктуации и специальных символов
        string = re.sub(r'[^a-z]', ' ', string)
        # Удаление повторяющихся пробелов, переносов строки, а также их удаление на концах строки
        return re.sub(r'\s+', ' ', string).lstrip().rstrip()
    else:
        # Замена отсутствующего значения пустой строкой
        return ''

Описанный процесс предобработки является достаточно интенсивным ввиду множества операций lookup и проверки символов слова. Для ускорения процесса предлагается использовать средства параллелизации задач. В модельном решении используется класс `Parallel` модуля `joblib` с параметрами `n_jobs=20` (количество параллельных задач) на основе мультипроцессинга (параметр `backend='multiprocessing'`). Другие бэкенды параллелизации `loky` и `threading` работают медленнее, как показал предварительный анализ.

In [4]:
df.iloc[:,5] = Parallel(n_jobs=20, backend='multiprocessing')(delayed(preprocess)(str(x)) for x in df.iloc[:,5])

В одном из модифицированных алгоритмов из задания предлагается включить текст из темы письма (колонка `subject` в датасете), который также нуждается в предобработке. Алгоритм предобработки остается таким же, как для содержания письма.

In [5]:
df.iloc[:, 4] = Parallel(n_jobs=20, backend='multiprocessing')(delayed(preprocess)(str(x)) for x in df.iloc[:,4])

Колонка `date` датасета была предварительно предобработана и приведена к стандартному виду ISO 8601 (без указания часового пояса). Для дальнейшего использования этой колонки в одном из измененных алгоритмов метка времени преобразуется в номер дня недели, начиная с 0, с помощью модуля `datetime`.

In [6]:
for i in range(len(df)):
    if pd.isnull(df.iat[i,1]):
        df.iat[i,1] = 0
    else:
        df.iat[i,1] = datetime.strptime(df.iat[i,1], "%Y-%m-%dT%H:%M:%S").weekday()

# Сохранение датасета

In [7]:
df.to_csv('Task_1_prepprocessed.csv', index=False)

## Пользовательская обработка тестового датасета

In [4]:
import os
nltk.download('omw-1.4')
with open(os.path.join(os.getcwd(), 'task_4_P_test_2.csv'), 'r') as file:
    body = file.read()

processed_body = preprocess(body=body)
with open(os.path.join(os.getcwd(), 'task_4_processed_test_2.csv'), 'w') as file:
    file.write(processed_body)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nshir\AppData\Roaming\nltk_data...
